## Modeling

In [1]:
import re
import time
import matplotlib.pyplot as plt
from pymongo import MongoClient
import pandas as pd
import numpy as np
import warnings
import random
import sqlite3
from sklearn.model_selection import train_test_split
import xgboost as xg
from sklearn.pipeline import Pipeline
from category_encoders import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
import shap

warnings.filterwarnings('ignore')

In [2]:
# SQL에서 데이터 가져오기
import os
url = os.path.join(os.getcwd(), 'flask_app/database/monitor.db')

conn = sqlite3.connect(url)
cur = conn.cursor()

query = "SELECT * FROM Monitor"
df = pd.read_sql(query, conn).drop('index', axis = 1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5116 entries, 0 to 5115
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품명     5116 non-null   object 
 1   화면크기    5116 non-null   object 
 2   최대 해상도  5116 non-null   object 
 3   최대 주사율  5116 non-null   int64  
 4   패널      5116 non-null   object 
 5   곡면형     5116 non-null   object 
 6   응답속도    5116 non-null   float64
 7   별점      5116 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 319.9+ KB


In [4]:
idx = {}

idx['idx_12'] = list(df[(df['최대 해상도'] == 'UHD') & (df['최대 주사율'] >= 144) & (df['응답속도'] <= 0.1)].index)
idx['idx_11'] = list(df[(df['최대 해상도'] == 'UHD') & (df['최대 주사율'] >= 144) & (df['응답속도'] > 0.1)].index)
idx['idx_10'] = list(df[(df['최대 해상도'] == 'UHD') & (df['최대 주사율'] < 144) & (df['응답속도'] <= 0.1)].index)
idx['idx_9'] = list(df[(df['최대 해상도'] == 'UHD') & (df['최대 주사율'] < 144) & (df['응답속도'] > 0.1)].index)

idx['idx_8'] = list(df[(df['최대 해상도'] == 'QHD') & (df['최대 주사율'] >= 144) & (df['응답속도'] <= 0.1)].index)
idx['idx_7'] = list(df[(df['최대 해상도'] == 'QHD') & (df['최대 주사율'] >= 144) & (df['응답속도'] > 0.1)].index)
idx['idx_6'] = list(df[(df['최대 해상도'] == 'QHD') & (df['최대 주사율'] < 144) & (df['응답속도'] <= 0.1)].index)
idx['idx_5'] = list(df[(df['최대 해상도'] == 'QHD') & (df['최대 주사율'] < 144) & (df['응답속도'] > 0.1)].index)

idx['idx_4'] = list(df[(df['최대 해상도'] == 'FHD') & (df['최대 주사율'] >= 144) & (df['응답속도'] <= 0.1)].index)
idx['idx_3'] = list(df[(df['최대 해상도'] == 'FHD') & (df['최대 주사율'] >= 144) & (df['응답속도'] > 0.1)].index)
idx['idx_2'] = list(df[(df['최대 해상도'] == 'FHD') & (df['최대 주사율'] < 144) & (df['응답속도'] <= 0.1)].index)
idx['idx_1'] = list(df[(df['최대 해상도'] == 'FHD') & (df['최대 주사율'] < 144) & (df['응답속도'] > 0.1)].index)



In [5]:
i=12
for value in idx.values():
    df.loc[value, '성능점수'] = [i] * len(value)
    i -= 1

df['성능점수'] = df['성능점수'].astype(int)

In [7]:
df.to_csv('/Users/kwanhoonseo/Desktop/sc3_pj/monitor.csv', sep = ',', index = False)

In [8]:
import psycopg2

conn = psycopg2.connect(
    host = "peanut.db.elephantsql.com",
    database = "pblabcdk",
    user = "pblabcdk",
    password = "HPGjhgcQnKL8_30d3RqYF574gmNSWDUj"
)

cur = conn.cursor()

In [18]:
# 성능점수가 포함된 데이터프레임 저장
"""
con = sqlite3.connect('/Users/kwanhoonseo/Desktop/sc3_pj/monitor.db')
df.to_sql('Monitor_score', con)
"""

In [19]:
df_md = df.drop(['상품명', '별점'], 1)
le = LabelEncoder()
train, test = train_test_split(df_md, test_size = 0.2, random_state = 42)
target = ['성능점수']

X_train, y_train = train.drop(target, 1), train[target]
X_test, y_test = test.drop(target, 1), test[target]

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [20]:
pipe = Pipeline(
    steps = [
        ('ord', OrdinalEncoder()),
        ('xgbc', xg.XGBClassifier(
            objective='multi:softmax',
            eval_metric = 'merror',
            num_class = 12,
            random_state = 2,
            n_estimators = 200,
            n_jobs = -1,
            max_depth = 7,
            learning_rate = 0.1
        ))
    ]
)

In [21]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('ord',
                 OrdinalEncoder(cols=['화면크기', '최대 해상도', '패널', '곡면형'],
                                mapping=[{'col': '화면크기',
                                          'data_type': dtype('O'),
                                          'mapping': 32     1
27     2
NaN   -2
dtype: int64},
                                         {'col': '최대 해상도',
                                          'data_type': dtype('O'),
                                          'mapping': FHD    1
UHD    2
QHD    3
NaN   -2
dtype: int64},
                                         {'col': '패널', 'data_type': dtype('O'),
                                          'mapping': VA     1
IPS    2
TN     3
NaN   -2
dtype: int64},
                                         {'col': '곡면형', 'data_type': dtype('O'),
                                          'mapping': 평면...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, inte

In [9]:
insert = pd.DataFrame(columns = ['화면크기', '최대 해상도', '최대 주사율', '패널', '곡면형', '응답속도'])
X_test = ['32', 'FHD', 75, 'VA', '평면', 0.4]

for i in range(6):
    insert.loc[0, insert.columns[i]] = X_test[i]

insert.iloc[0]

화면크기       32
최대 해상도    FHD
최대 주사율     75
패널         VA
곡면형        평면
응답속도      0.4
Name: 0, dtype: object

In [10]:
score = le.inverse_transform(pipe.predict([X_test.iloc[0]]))[0]

max_star = df[df['성능점수'] == score]['별점'].max()
df[(df['성능점수'] == score) & (df['별점'] == max_star)].상품명

NameError: name 'le' is not defined

In [27]:
import pickle

with open('model/monitor_score.pkl','wb') as pickle_file:
    pickle.dump(pipe, pickle_file)
    
with open('model/labelencoding.pkl','wb') as pickle_file:
    pickle.dump(le, pickle_file)